<a href="https://colab.research.google.com/github/SUB15LUIZ/Projetos_Luiz/blob/main/Consulta_SGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
## Extração Informações Sistema Gerenciador de Séries Temporais - Bacen

import requests
import pandas as pd
import os
from datetime import datetime, timedelta

#Conexão com o google drive
from google.colab import drive
drive.mount('/content/drive')

#Variaveis de caminho
caminho_entrada = "/content/drive/MyDrive/Colab Notebooks/Consulta_Bacen/Lista_de_series.xlsx"
saida_dir = "/content/drive/MyDrive/Colab Notebooks/Consulta_Bacen"


def consultar_SGS(codigos_periodicidade: list, meses_retroativos=6):
  # Dataframe final extração
  df_base = None
  # Dataframe da data utilizada no processo
  datas = None
  # DataFrame para registrar erros de consulta
  df_erros = pd.DataFrame(columns=['codigo', 'periodicidade', 'erro', 'detalhes'])

  for item in codigos_periodicidade:
    cod_item = item['codigo']
    periodicidade = item['periodicidade'].lower()
    hoje = datetime.now().replace(day=1)

    # Verifica qual a periodicidade e monta o calendário de datas
    if periodicidade == "mensal":
        dt_ini = hoje - timedelta(days=meses_retroativos * 30)
        datas = pd.date_range(dt_ini, hoje, freq='MS')

    elif periodicidade == "trimestral":
        dt_ini = hoje - timedelta(days=meses_retroativos * 30 * 3)
        datas = pd.date_range(dt_ini, hoje, freq='QS')

    elif periodicidade == "diária":
        hoje = datetime.now()
        dt_ini = hoje - timedelta(days=meses_retroativos * 30)
        datas = pd.date_range(dt_ini, hoje, freq='D')

    else:
        print(f"Periodicidade '{periodicidade}' não tratada para código {cod_item}. Pulando...")
        continue

    # Cria DataFrame temporário com as datas
    df_temp = pd.DataFrame({'data': datas.strftime('%d/%m/%Y')})
    dt_ini_str = dt_ini.strftime('%d/%m/%Y')
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{cod_item}/dados?formato=json&dataInicial={dt_ini_str}"

    try:
        resposta = requests.get(url)
        if resposta.status_code == 200 and resposta.text.strip():
            dados = resposta.json()
            df_api = pd.DataFrame(dados)
            df_api.rename(columns={'valor': str(cod_item)}, inplace=True)
            df_temp = pd.merge(df_temp, df_api, on='data', how='left')
            df_base = df_temp if df_base is None else pd.merge(df_base, df_temp, on='data', how='outer')
        else:
          df_erros.loc[len(df_erros)] = {
          'codigo': cod_item,
          'periodicidade': periodicidade,
          'erro': f'HTTP {resposta.status_code}',
          'detalhes': resposta.text.strip()[:100]
          }
    except Exception as e:
        df_erros.loc[len(df_erros)] = {
          'codigo': cod_item,
          'periodicidade': periodicidade,
          'erro': 'Exception',
          'detalhes': str(e)
          }

  return df_base, df_erros

#if __name__ = "__main__":

  # import os
  # #Conexão com o google drive
  # from google.colab import drive
  # drive.mount('/content/drive')

  # caminho_entrada = "/content/drive/MyDrive/Colab Notebooks/Consulta_Bacen/Lista_de_series.xlsx"
  # saida_dir = "/content/drive/MyDrive/Colab Notebooks/Consulta_Bacen"

# Ler o arquivo Excel
df_codigos = pd.read_excel(caminho_entrada)

df_codigos_mensal = df_codigos[df_codigos['Frequencia_dado'] == 'Mensal']
df_codigos_trimestral = df_codigos[df_codigos['Frequencia_dado'] == 'Trimestral']

codigos_periodicidade_mensal = df_codigos_mensal[['Codigo', 'Frequencia_dado']].rename(
    columns={"Codigo": "codigo", "Frequencia_dado": "periodicidade"}
).to_dict(orient='records')

codigos_periodicidade_trimestral = df_codigos_trimestral[['Codigo', 'Frequencia_dado']].rename(
    columns={"Codigo": "codigo", "Frequencia_dado": "periodicidade"}
).to_dict(orient='records')

#print (codigos_periodicidade_mensal)
#print (codigos_periodicidade_trimestral)

#Chama a consulta e guarda em um dataframe
saida_dados_mensal,df_erros_mensal = consultar_SGS(codigos_periodicidade_mensal)
saida_dados_trimestral,df_erros_trimestral = consultar_SGS(codigos_periodicidade_trimestral)
df_erros = pd.concat([df_erros_mensal, df_erros_trimestral], ignore_index=True)

# Criar dicionário para renomeação pós-processamento
mapa_nomes_mensal = df_codigos_mensal.set_index('Codigo')['Nome_Ajustado'].to_dict()
mapa_nomes_trimestral = df_codigos_trimestral.set_index('Codigo')['Nome_Ajustado'].to_dict()

# Renomea colunas para melhor utilização
renomear_mensal = {str(k): f"{k}_{v}" for k, v in mapa_nomes_mensal.items()}
renomear_trimestral = {str(k): f"{k}_{v}" for k, v in mapa_nomes_trimestral.items()}

saida_dados_mensal.rename(columns=renomear_mensal, inplace=True)
saida_dados_trimestral.rename(columns=renomear_trimestral, inplace=True)



#Exportar para um csv
saida_dados_mensal.to_csv(os.path.join(saida_dir, "series_unificadas_mensal.csv"), index=False, encoding='utf-8-sig')
saida_dados_trimestral.to_csv(os.path.join(saida_dir, "series_unificadas_trimestral.csv"), index=False, encoding='utf-8-sig')

if not df_erros.empty:
  df_erros.to_csv(os.path.join(saida_dir, "erros_consulta.csv"), index=False, encoding='utf-8-sig')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
